# Tugas 2

**DISKRITISASI**

Deskritisasi atau binning digunakan untuk mengubah atribut numerik menjadi atribut kategorikal. Perubahan tersebut dilakukan dengan mengkategorikan atribut numerik menjadi beberapa tingkatan atribut kategorikal.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset_url="https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"
data=pd.read_csv(dataset_url)

In [ ]:
data[['sepal_length']]

,sepal_length
0,5.1
1,4.9
2,4.7
3,4.6
4,5.0
...,...
145,6.7
146,6.3
147,6.5
148,6.2


In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')

In [ ]:
data['sepal_length']=est.fit_transform(data[['sepal_length']])

In [ ]:
data['sepal_length']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
145    2.0
146    2.0
147    2.0
148    2.0
149    1.0
Name: sepal_length, Length: 150, dtype: float64

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
data['petal_length']=est.fit_transform(data[['petal_length']])

In [ ]:
data['petal_length']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
145    2.0
146    2.0
147    2.0
148    2.0
149    2.0
Name: petal_length, Length: 150, dtype: float64

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
data['petal_width']=est.fit_transform(data[['petal_width']])

In [ ]:
data['petal_width']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
145    3.0
146    3.0
147    3.0
148    3.0
149    2.0
Name: petal_width, Length: 150, dtype: float64

**Entropy**

Entropi adalah nilai informasi yang menyatakan ukuran ketidakpastian(impurity) dari attribut dari suatu kumpulan obyek data dalam satuan bit

membuat sampel untuk dianalisis

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset_url="https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"
data=pd.read_csv(dataset_url)

In [ ]:
sampel = data[["sepal_length"]]
sampel.describe()

,sepal_length
count,150.000000
mean,5.843333
std,0.828066
min,4.300000
25%,5.100000
50%,5.800000
75%,6.400000
max,7.900000


membuat category random untuk semua data

In [ ]:
np.random.seed(0)
sampel["category"] = np.where(np.random.choice(2, sampel.shape[0]) < 1, "A", "B")
sampel

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sepal_length,category
0,5.1,A
1,4.9,B
2,4.7,B
3,4.6,A
4,5.0,B
...,...,...
145,6.7,A
146,6.3,B
147,6.5,B
148,6.2,B


membuat fungsi getOverCategory yang digunakan untuk menghitung data keseluruhan yang nantinya digunakan untuk menghitung entropy

In [ ]:
def getOverCategory(col):
  group = sampel.loc[:, :].groupby("category").count()
  a = group.loc["A", col]
  b = group.loc["B", col]
  return (a, b, a+b)

fungsi splitter digunakan untuk membuat split antara value yang telah ditentukan lalu mengembalikan data yang telah dipisahkan

In [ ]:
def splitter(value:float, col:str)->tuple:
  # get data less and greater from value
  less = sampel[sampel[col] <= value]
  greater = sampel[sampel[col] > value]

  # calculate into category for each data
  less_group = less.loc[:, :].groupby("category").count()
  greater_group = greater.loc[:, :].groupby("category").count()

  # get value based on category
  less_category_A = less_group.loc["A", col] 
  less_category_B = less_group.loc["B", col] 
  greater_category_A = greater_group.loc["A", col] 
  greater_category_B = greater_group.loc["B", col] 

  return (
      [less_category_A, less_category_B, less_category_A + less_category_B],
      [greater_category_A, greater_category_B, greater_category_A + greater_category_B]
  )

Membuat fungsi entropy untuk mencari nilai entropy

Rumus Mencari Entropy :
$$
Entropy (D_{1}) = - \sum_{i=1}^{m} pi \log_{2} pi
$$


In [9]:
def entropy(d):
  r1 = (d[0] / d[2]) * np.log2(d[0] / d[2]) 
  r2 = (d[1] / d[2]) * np.log2(d[1] / d[2]) 
  return np.sum([r1, r2]) * -1

Membuat fungsi info

In [10]:
def info(d):
  r1 = (d[0][2] / sampel.shape[0]) * entropy(d[0])
  r2 = (d[1][2] / sampel.shape[0]) * entropy(d[1])
  return r1 + r2

Mmebuat fungsi gain untuk menghitung selisih antara entropy awal dengan yang baru.

Rumus Mencari Gain :
$$
Gain(E_{new}) = (E_{initial}) - (E_{new})
$$

In [11]:
def gain(Einitial, Enew):
  return Einitial - Enew

Membuat DInitial

In [12]:
D = getOverCategory("sepal_length")
entropy_d = entropy(D)
print(D)
print(entropy_d)

(68, 82, 150)
0.993707106604508


Melakukan beberapa tes split untuk mencari hasil dan informasi yang terbaik

Tes Pertama : Split 1:4.4

In [13]:
split1  = splitter(4.4, "sepal_length")
info_split1 = info(split1)
gain(entropy_d, info_split1)

0.003488151753460178

Tes Kedua : Split 2:5.5

In [14]:
split2  = splitter(5.5, "sepal_length")
info_split2 = info(split2)
gain(entropy_d, info_split2)

0.012302155146638905

Tes Ketiga : Split 3:7.0

In [15]:
split3  = splitter(7.0, "sepal_length")
info_split3 = info(split3)
gain(entropy_d, info_split3)

0.0005490214732508658

Dari seluruh hasil percobaan tes split yang telah dilakukan, maka diperoleh hasil split terbaik adalah split 3 yang mwemberikan keuntungan informasi sebesar 0.0005490214732508658, karena hasil tes split 3 memiliki nilai split yang terendah.